In [205]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder


#reads in the data
data = pd.read_csv('movie_metadata.csv')

#removes the values that aren't numeric
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews']

#makes the new data set without the to_drop colums
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 
   
print('Done')

Done


# Changing Genres into individual Columns

In [206]:
genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

{'History', 'Film-Noir', 'Comedy', 'Horror', 'Drama', 'Thriller', 'Fantasy', 'Sport', 'Adventure', 'Documentary', 'Action', 'Western', 'Music', 'Reality-TV', 'Sci-Fi', 'News', 'Game-Show', 'Mystery', 'Romance', 'Biography', 'Animation', 'War', 'Musical', 'Crime', 'Family', 'Short'}


In [207]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

del movie_data['genres']
#print(genres_dict['Short'])

for genre in genres_set:
    series = pd.Series(genres_dict[genre])
    movie_data[genre] = series

#print(movie_data['Short'].values)    
print('Done')

Done


# Deleting NaN's

In [208]:
NA_THRESH = 4

#deletes the row when there are more or equal to the threshod number of NaN's
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data

#returns length
movie_data = remove_too_many_NaN(movie_data, NA_THRESH)

Length before deletion: 5043
Length after deletion: 4900


# Changing Color to Numeric

In [209]:
#replaces the color value to 1
movie_data.color = movie_data.color.replace(to_replace = 'Color', value = 1)
movie_data.color = movie_data.color.replace(to_replace = 'NaN', value = 1)

#replaces the NaN or black and white values to 0
for item in movie_data.color:
    if item != 1:
        movie_data.color = movie_data.color.replace(to_replace = item, value = 0)
        
#movie_data.color = new_color_column 
#print(movie_data['color'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Changing Country to Numeric

In [210]:
#replaces the USA values to 1
movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)
movie_data.country = movie_data.country.replace(to_replace ='NaN', value = 1)
#replaces the NaN or non USA values to 0
for item in movie_data.country:
    if item != 1:
        movie_data.country = movie_data.country.replace(to_replace = item, value = 0)
       
#print(movie_data['country'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None

print('Done')

Done


# Changing Languange to Numeric

In [211]:
#replaces the English values to 1
movie_data.language = movie_data.language.replace(to_replace = 'English', value = 1)
movie_data.language = movie_data.language.replace(to_replace = 'NaN', value = 1)

#replaces the other values to 0
for item in movie_data.language:
    if item != 1:
        movie_data.language = movie_data.language.replace(to_replace = item, value = 0)
        
#print(movie_data['language'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Chaning Content_Rating to Numeric

In [212]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

movie_data = content_to_numerical(movie_data)
print(movie_data)

      actor_1_facebook_likes  actor_2_facebook_likes  actor_3_facebook_likes  \
0                     1000.0                   936.0                   855.0   
1                    40000.0                  5000.0                  1000.0   
2                    11000.0                   393.0                   161.0   
3                    27000.0                 23000.0                 23000.0   
5                      640.0                   632.0                   530.0   
6                    24000.0                 11000.0                  4000.0   
7                      799.0                   553.0                   284.0   
8                    26000.0                 21000.0                 19000.0   
9                    25000.0                 11000.0                 10000.0   
10                   15000.0                  4000.0                  2000.0   
11                   18000.0                 10000.0                   903.0   
12                     451.0            

# Replacing NaNs with averages

In [213]:
def replace_NaNs(col):
    # compute average
    avg = np.sum(col) / (len(col) - np.sum(col.isnull()))
    print(avg)
    
    # replace NaNs with average
    col = col.fillna(value=avg)
    return col

pd.options.mode.chained_assignment= None

to_replace_NaNs = ['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 
               'aspect_ratio', 'budget', 'cast_total_facebook_likes', 'director_facebook_likes', 'gross', 'title_year'
                   , 'duration', 'facenumber_in_poster', 'content_rating']
for column in to_replace_NaNs:
    movie_data[column] = replace_NaNs(movie_data[column])


6648.62979592
1684.1759902
653.779595175
2.1284344634
39892924.8304
9849.74408163
692.662510221
48514997.6517
2002.39901881
108.243568804
1.36272504092
13.6882617062


# Delete columns with too little 1's

In [214]:
for genre in genres_set:
    print(genre, ':')
    counter = 0
    for value in movie_data[genre]:
        if value == 1.0:
            counter+=1
    print(counter)
    
weghalen = ['Game-Show', 'News', 'Reality-TV', 'Short', 'Film-Noir']

for name in weghalen:
    del movie_data[name]
    
#print(movie_data)

History :
203
Film-Noir :
6
Comedy :
1824
Horror :
550
Drama :
2509
Thriller :
1375
Fantasy :
593
Sport :
181
Adventure :
910
Documentary :
111
Action :
1131
Western :
97
Music :
213
Reality-TV :
0
Sci-Fi :
596
News :
3
Game-Show :
0
Mystery :
475
Romance :
1088
Biography :
292
Animation :
233
War :
208
Musical :
132
Crime :
858
Family :
532
Short :
3


# Trainset en testset

In [215]:
ratings = movie_data['imdb_score'].values
del movie_data['imdb_score']

X = movie_data.values
X_std = StandardScaler().fit_transform(X)

number_of_samples = len(ratings)
np.random.seed(0)
random_indices = np.random.permutation(number_of_samples)
num_training_samples = int(number_of_samples*0.6)
num_validation_samples = int(number_of_samples*0.2)+num_training_samples

movie_training = X_std[random_indices[:num_training_samples]]
ratings_training = ratings[random_indices[:num_training_samples]]

movie_validation = X_std[random_indices[num_training_samples:num_validation_samples]]
ratings_validation = ratings[random_indices[num_training_samples:num_validation_samples]]

movie_test = X_std[random_indices[num_validation_samples:]]
ratings_test = ratings[random_indices[num_validation_samples:]]

ratings_training = list(ratings_training)

print(len(ratings_training))
print(len(ratings_validation))
print(len(ratings_test))

2940
980
980


# Leren

In [216]:
def relative_error(y_predict, y):
    
    error = 0

    for i in range(len(y)):
        error += (abs(y_predict[i]-y[i]))/y[i]
    training_error = error/len(y)*100
    
    return training_error

### neural networks

In [217]:
from sklearn.neural_network import MLPRegressor

neural = MLPRegressor(activation='logistic', max_iter = 500)

neural.fit(movie_training, ratings_training)
y_neural_1 = neural.predict(movie_training)
y_neural_2 = neural.predict(movie_test)

training_error = relative_error(y_neural_1, ratings_training)

print("Train error = "+'{}'.format(training_error)+" percent"+" in neural network algorithm")

test_error = relative_error(y_neural_2,ratings_test)

print("Test error = "'{}'.format(test_error)+" percent"+" in neural network algorithm")

Train error = 10.481533827623254 percent in neural network algorithm
Test error = 10.977370064632183 percent in neural network algorithm


### k-nearest neighbors

In [218]:
regressor = KNeighborsRegressor()
regressor.fit(movie_training, ratings_training)
y_predict_train = regressor.predict(movie_training)
y_predict_train = list(y_predict_train)

train_error = relative_error(y_predict_train,ratings_training)

print("Train error = "+'{}'.format(training_error)+" percent"+" in knn algorithm")

y_predict_test = regressor.predict(movie_test)
y_predict_test = list(y_predict_test)

test_error = relative_error(y_predict_test, ratings_test)

print("Test error = "'{}'.format(test_error)+" percent"+" in knn algorithm")

Train error = 10.481533827623254 percent in knn algorithm
Test error = 12.673746424295157 percent in knn algorithm


### Support vector machine

In [219]:
from sklearn.svm import SVR, NuSVR, LinearSVR
svr = SVR()
svr.fit(movie_training, ratings_training)
y_train = svr.predict(movie_training)
y_train = list(y_train)

train_error = relative_error(y_train, ratings_training)

print ("Train error = "+'{}'.format(train_error))

y_test = svr.predict(movie_test)
y_test = list(y_test)

test_error = relative_error(y_test, ratings_test)

print ("Test error = "+'{}'.format(test_error))

Train error = 9.150128853631944
Test error = 10.858612788391119


### Random Forest

In [220]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

random_forest = RandomForestRegressor()
random_forest.fit(movie_training, ratings_training)
y_predict_train = regressor.predict(movie_training)
y_predict_train = list(y_predict_train)

test_error = relative_error(y_predict_train, ratings_training)

print("Train error = "+'{}'.format(training_error)+" percent"+" in random forest")

y_predict_test = regressor.predict(movie_test)
y_predict_test = list(y_predict_test)

test_error = relative_error(y_predict_test, ratings_test)

print("Test error = "'{}'.format(test_error)+" percent"+" in random forest")

Train error = 10.481533827623254 percent in random forest
Test error = 12.673746424295157 percent in random forest


### Bagging regressor

In [221]:
#regressor  = BaggingRegressor(MLPRegressor(max_iter = 300))
regressor  = BaggingRegressor(SVR())
regressor.fit(movie_training, ratings_training)
y_predict_train = regressor.predict(movie_training)
y_predict_train = list(y_predict_train)

test_error = relative_error(y_predict_train ,ratings_training)

print("Train error = "+'{}'.format(training_error)+" percent"+" in random forest")

y_predict_test = regressor.predict(movie_test)
y_predict_test = list(y_predict_test)

test_error = relative_error(y_predict_test, ratings_test)

print("Test error = "'{}'.format(test_error)+" percent"+" in random forest")

Train error = 10.481533827623254 percent in random forest
Test error = 10.843104428460116 percent in random forest


# Validation

### neural networks

In [223]:
neural_1 = MLPRegressor(hidden_layer_sizes = 5, max_iter = 1000)
neural_2 = MLPRegressor(hidden_layer_sizes = 20, max_iter = 1000)
neural_3 = MLPRegressor(hidden_layer_sizes = 100, max_iter = 1000)